### Meshwork: linking meshes, skeletons, and annotations

There are many ways to describe neuroanatomy. We often work with three of them:
* Meshes. Meshes provide high resolution 3d structure of the surface of a cell. This is important for understanding the fine details of a neuron, like how long spines are or how the shape of a bouton.
* Skeletons. Skeletons provide a tree-like topological structure of a cell. They make it easy to ask questions about proximal/distalness or branch points, are much faster to compute on, and are the typical way neurons are described in most of neuroscience.
* Annotations. Annotations decorate the structure of a neuron and can indicate all sorts of things, from synapses to soma location to the base of an axon.

The `Meshwork` class helps work interoperably with these three views of the same neuron and gets rid of a lot of the tedious computation one often needs to do when navigating back and forth between these representations.

#### 1. Building a Meshwork object

The core of a Meshwork object is, as the name suggests, the mesh. Meshes are one of the fundamental objects in the Connectome Annotation Versioning Engine, and every root id has a unique mesh and can be downloaded through cloudvolume. We call this global version, which is the same for everyone, the *Base Mesh*.

However, the Base Mesh often has artifacts that cause problems for analysis. For example, internal structures like vesicles or the nucleus can be segmented separately and have internal mesh vertices or artifacts in across several sections of imagery can cause a continous branch to have a gap. While these don't usually cause obvious problems, they present technical problems. Internal mesh vertices can 'grab' annotations like synapses, and gaps make it impossible to naively compute path distances.

Because of this, we need to define a derived mesh that will be used to build continuous skeletonizations and anchor annotations. We call this the *Anchor Mesh*. The Anchor Mesh usually has internal vertices removed and gaps bridged. However, it's important to note that there are choices that go into defining this mesh, and thus it is not a universal object.

Building a Meshwork object starts with creating an Anchor Mesh. Here, we filter the original mesh to keep only the vertices in the largest connected component, which is sufficient to clean up the example mesh.

In [ ]:
from meshparty import meshwork
from meshparty import trimesh_io, trimesh_vtk, skeletonize, mesh_filters
import numpy as np
import pandas as pd
import os

# Specify the base mesh:

oid = 648518346349539789

mm = trimesh_io.MeshMeta()
mesh_base = mm.mesh(filename=f'meshes/{oid}.h5')

# Filter out mesh vertices that are not in the largest connected component
in_comp = mesh_filters.filter_largest_component(mesh_base)
mesh_anchor = mesh_base.apply_mask(in_comp)

# The basic Meshwork takes the anchor mesh
nrn = meshwork.Meshwork(mesh_anchor, seg_id=oid)

#### Annotations 

The first useful feature of a Meshwork object is dynamic annotation management. Annotations must be in the form of pandas dataframes. A given annotation has a name, a dataframe, and can be either 'anchored' to the mesh or not. For anchored dataframes, there must be a specified point column name. Each row gets associated with the closest mesh vertex to its point in this column. Unanchored annotations don't have attached mesh vertices, but can be useful for tracking information.

In [ ]:
syn_in_df = pd.read_hdf('syn.h5', 'post')
nrn.add_annotations('syn_in', syn_in_df, point_column='ctr_pt_position')

Annotations are found under the `.anno` property. Each annotation can be accessed either as a property or as a dictionary with the name as a key. 

An annotation has a number of different properties:
* .df : A length-n dataframe. Note than an anchored annotation has a new column, `mesh_index`, that specifies the index of the mesh index it is anchored to.

In [ ]:
nrn.anno.syn_in.df.head()

* .voxels : An n x 3 array of voxel locations for each annotation point.

In [ ]:
nrn.anno.syn_in.voxels

* .points : An n x 3 array of point locations for each annotation point in the same units as the mesh vertices.

In [ ]:
nrn.anno.syn_in.points

* .mesh_index : An array of mesh indices for each annotation point. (In fact, this array is actually a MeshIndex, but it can be used like a standard numpy array.

In [ ]:
nrn.anno.syn_in.mesh_index

For example, we can use the point property to easily make a visualization of the synapses.

In [ ]:
syn_actor = trimesh_vtk.point_cloud_actor(nrn.anno.syn_in.points, size=800, color=(0.2, 0.9, 0.9))
mesh_actor = trimesh_vtk.mesh_actor(nrn.mesh, opacity=1, color=(0.7, 0.7, 0.7))
trimesh_vtk.render_actors([mesh_actor, syn_actor])

Additional annotations can keep being added.

Here, we want the `soma_pt` annotation to only have the approximate center of the soma, but don't want to link it to any paritcular mesh index. We thus set `anchored=False`. This will keep a mesh index from being computed and prevent any filtering, although voxel and point locations can still be accessed.

In [ ]:
syn_out_df = pd.read_hdf('syn.h5', 'pre')
soma_df = pd.read_hdf('syn.h5', 'soma')

nrn.add_annotations('syn_out', syn_out_df, point_column='ctr_pt_position')
nrn.add_annotations('soma_pt', soma_df.query('pt_root_id == @oid').copy(), point_column='pt_position', anchored=False)
nrn.anno

#### Masks
Boolean masks work on meshwork objects, similar to meshes alone, but in this case they apply to the mesh vertices and annotations together. 

In [ ]:
# Make a mask object for mesh vertices within 30000 nm of a particular synapse
close_to_point_mask = mesh_filters.filter_spatial_distance_from_points(nrn.mesh,
                                                                       nrn.anno.syn_in.points[10],
                                                                       30000)

# Apply the mask and visualize the mesh as before (we also show the anchor mesh for comparison)
nrn.apply_mask(close_to_point_mask)

syn_actor = trimesh_vtk.point_cloud_actor(nrn.anno.syn_in.points, size=800, color=(0.2, 0.9, 0.9))
mesh_actor = trimesh_vtk.mesh_actor(nrn.mesh, opacity=1, color=(0.7, 0.7, 0.7))
mesh_base_actor = trimesh_vtk.mesh_actor(mesh_anchor, opacity=0.2, color=(0.7, 0.7, 0.7))
trimesh_vtk.render_actors([mesh_actor, syn_actor, mesh_base_actor])

Unlike meshes, meshwork objects retain the memory of the Anchor Mesh and can always be reset.

In [ ]:
print(f'There are {len(nrn.anno.syn_in)} input synapses before reset.')
nrn.reset_mask()
print(f'There are {len(nrn.anno.syn_in)} input synapses after reset.')

Sometimes you want to query what annotations are within a given node mask and not change the whole object. We can accomplish this with the `filter_query` function like so:

In [ ]:
len(nrn.anno.syn_in.filter_query(close_to_point_mask).df)

#### Skeletons : Adding directed topology

A meshwork class can also build a skeleton from their Anchor Mesh. Much of the utility of a meshwork object comes from being able to link extremely fast skeleton operations like finding a path between two points to mesh structures and annotations.

In [ ]:
nrn.skeletonize_mesh(soma_pt=nrn.anno.soma_pt.points[0], soma_thresh_distance=8500)

The mesh now has a `.skeleton` property. For example, we can quickly compute the total cable length of the mesh in microns:

In [ ]:
nrn.skeleton.path_length()/1000

However, it would be confusing for us to always go back and forth between meshes and skeletons. In general, we will stick to dealing with mesh vertices. Inside the Meshwork object the conversion from mesh to skeleton vertices and back again is handled automatically. For example, `nrn.root` returns a mesh index associated with the skeleton root, which here is set to the soma position.

Here, let's look at the child nodes of the root, which should be the base of each dendritic and axonal process. For each index, we can ask for the skeleton downstream of that point.

In [ ]:
# Get all children of the root node, which is the soma.
branch_starts = nrn.child_index(nrn.root)
# Let's just pick one of these
mesh_downstream = nrn.downstream_of(branch_starts[2])
mesh_downstream

Functions like `path_length` exist at the meshwork level where they expect mesh indices.

In [ ]:
nrn.path_length(mesh_downstream) / 1000

### JointMeshIndex and JointSkeletonIndex

You'll note that the type of `mesh_downstream` is not an array, but a `JointMeshIndex`. This is a class that contains methods to convert mesh indices to other representations, such as boolean masks and skeleton indices.

MeshIndices have a number of conversion functions for different situations.

In [ ]:
# to_mesh_index just returns the same values
mesh_downstream.to_mesh_index

In [ ]:
# to_mesh_mask returns a boolean mask with True at the location of the indices
mesh_downstream.to_mesh_mask

In [ ]:
# to_skel_index returns the skeleton indices for each. Note: This does not give a 1-1 correspondance between indices.
mesh_downstream.to_skel_index

In [ ]:
# to_skel_index returns the skeleton indices for each. Has a -1 if no map is available for the index.
mesh_downstream.to_skel_index_padded

Skeleton indices are like mesh indices, but because there is a 1-to-many mapping between skeleton vertices and mesh vertices they have a few more options.

In [ ]:
# SkeletonIndexs are similar, but for meshes
# Let's make one from the downstream values.
skinds = np.unique( mesh_downstream.to_skel_index )

# to_mesh_index returns the exact mesh index
skinds.to_mesh_index

In [ ]:
# to_mesh_mask returns the mesh mask for the skeleton indices
skinds.to_mesh_mask

In [ ]:
# to_mesh_region returns a list of MeshIndex arrays for each element of the skeleton indices
skinds.to_mesh_region

A list of indices can be converted into a MeshIndex or SkeletonIndex through 
`nrn.MeshIndex` or `nrn.SkeletonIndex` methods.

In [ ]:
minds = nrn.MeshIndex([1,2,3])
minds.to_skel_index

#### Putting operations together
As an example, let's look at the synapses per unit length for each branch by going through each branch off the root and computing its total path length and the number of synaptic inputs and outputs on it.

In [ ]:
syn_in_on_branch = []
syn_out_on_branch = []
len_of_branch = []

branch_starts = nrn.child_index(nrn.root)

for bp in branch_starts:
    mesh_downstream = nrn.downstream_of(bp)
    syn_in_on_branch.append(len(nrn.anno.syn_in.filter_query(mesh_downstream.to_mesh_mask).df))
    syn_out_on_branch.append(len(nrn.anno.syn_out.filter_query(mesh_downstream.to_mesh_mask).df))
    len_of_branch.append(nrn.path_length(mesh_downstream))

syn_in_on_branch = np.array(syn_in_on_branch)
syn_out_on_branch = np.array(syn_out_on_branch)
len_of_branch = np.array(len_of_branch)/1000  # in microns

syn_in_per_micron = syn_in_on_branch/len_of_branch
syn_out_per_micron = syn_out_on_branch/len_of_branch

pd.DataFrame({'Input density': syn_in_per_micron, 'Output density': syn_out_per_micron})

Let's visualize the branch with the highest density of inputs:

In [ ]:
nrn.reset_mask()
branch_ind = np.argmax(syn_in_per_micron)
mesh_downstream = nrn.downstream_of(branch_starts[branch_ind])
syn_points_downstream = nrn.anno.syn_in.filter_query(mesh_downstream.to_mesh_mask).points

syn_actor = trimesh_vtk.point_cloud_actor(syn_points_downstream, size=800, color=(0.2, 0.8, 0.8))
mesh_actor = trimesh_vtk.mesh_actor(nrn.mesh, opacity=1, color=(0.7, 0.7, 0.7))
trimesh_vtk.render_actors([syn_actor, mesh_actor])

#### Loading and saving

We can loading and save the whole meshwork object. Note that it saves the original anchor mesh (*not* the base mesh) and all annotations, but does apply the same mask.

In [ ]:
filename = f"{nrn.seg_id}_meshwork.h5"
nrn.save_meshwork(filename, overwrite=True)

In [ ]:
nrn2 = meshwork.load_meshwork(filename)
np.all( nrn.branch_points == nrn2.branch_points )

#### More skeleton operations

In general, operations that consider paths along the neurite or have a sense of distal/proximal have these skeleton-like meshwork functions.

End points can be accessed like branch points:

In [ ]:
nrn.end_points

Segments are regions between branch and end points. Here, let's compute them and highlight the segment with the most input synapses.

In [ ]:
# compute_all_segments and look at two of them
mesh_segs = nrn.segments()
mesh_segs[0:2]

In [ ]:
# Count inputs 
nsyn_in = []
for seg in mesh_segs:
    nsyn_in.append(len(nrn.anno.syn_in.filter_query(seg.to_mesh_mask).df))

clrs = np.array( [(0.3, 0.3, 0.3), (0.8, 0.2, 0.2)] )
seg = mesh_segs[np.argmax(nsyn_in)]
mesh_colors = clrs[seg.to_mesh_mask.astype(int)]

ma = trimesh_vtk.mesh_actor(nrn.mesh, vertex_colors=mesh_colors, opacity=1)
trimesh_vtk.render_actors([ma])

The geodesic distance between mesh vertices along the skeleton is computed with `distance_between`. Here, we compute the distribution of closest distances between output synapses. The `distance_to_root` function is convenient wrapper for finding the distance to the root node.

In [ ]:
d_out = nrn.distance_between(nrn.anno.syn_out.mesh_index, nrn.anno.syn_out.mesh_index)
d_out

In [ ]:
# Remove the trivial 0 along the diagonal
d_out_nodiag = d_out + np.diag(np.inf*np.ones(len(d_out)))
intersynapse_d = np.min(d_out_nodiag, axis=1)

In [ ]:
import matplotlib.pyplot as plt
_ = plt.hist(intersynapse_d/1000, bins=np.arange(0,40, 2))

The mesh points along the path between mesh points can be computed with `path_between`. Here, we plot synapse sizes as a function of distance to root along a single path from one end point to root.

In [ ]:
path_to_root = nrn.path_between(nrn.end_points[5], nrn.root)
syn_on_path = nrn.anno.syn_in.filter_query(path_to_root.to_mesh_mask)

fig, ax = plt.subplots(figsize=(5, 3), dpi=100)
ax.plot(nrn.distance_to_root(syn_on_path.mesh_index)/1000, syn_on_path.df['size'], '.')
_ = ax.set_xlabel('Dist to root ($\mu m)')
_ = ax.set_ylabel('Synapse size')

### Using the skeleton to split axon/dendrite



In [ ]:
from meshparty.meshwork import algorithms

is_axon, qual = algorithms.split_axon_by_synapses(nrn, 
                                                  nrn.anno.syn_in.mesh_index,
                                                  nrn.anno.syn_out.mesh_index,
                                                  )

# split_axon_by_synapses returns a skeleton mask, which we want to convert to a SkeletonIndex 
is_axon_skel = nrn.SkeletonIndex(np.flatnonzero(is_axon))

In [ ]:
clrs = np.array([[0.7, 0.7, 0.7], [0.8, 0.2, 0.3]])
mesh_color = clrs[is_axon_skel.to_mesh_mask.astype(int)]

ma = trimesh_vtk.mesh_actor(nrn.mesh, vertex_colors=mesh_color, opacity=1)
trimesh_vtk.render_actors([ma])

### Linear density

This function computes the density of annotations (e.g. synapses) along a moving window across the branches of a neuron, normalized by cable length.

In [ ]:
# Let us look at linear synapse density only on the dendrites
nrn.apply_mask(np.invert(is_axon_skel.to_mesh_mask))

In [ ]:
rho = nrn.linear_density(nrn.anno.syn_in.mesh_index, 2500, normalize=True, exclude_root=True)

ma = trimesh_vtk.mesh_actor(nrn.mesh, vertex_colors=(1000*rho-0.5), opacity=1)
trimesh_vtk.render_actors([ma])